In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
torch.cuda.set_device(0)

Path to Pascal VOC, we'll be using 2007

In [3]:
PATH = Path('/home/sean/data/VOCdevkit/VOC2007')
list(PATH.iterdir())

[PosixPath('/home/sean/data/VOCdevkit/VOC2007/ImageSets'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/pascal_train2007.json'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/pascal_val2007.json'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/Annotations'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/annotations_cache'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/pascal_test2007.json'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/SegmentationObject'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/SegmentationClass'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/results'),
 PosixPath('/home/sean/data/VOCdevkit/VOC2007/JPEGImages')]

Inspect the dataset

In [15]:
train_j = json.load((PATH/'pascal_train2007.json').open())
print(train_j.keys())
IMAGES,ANNOTATIONS,CATEGORIES = ['images', 'annotations', 'categories']
annotations = train_j['annotations']
print(annotations[0], '\n-----')
print(train_j[IMAGES][0], '\n-----')
print(train_j[CATEGORIES])


dict_keys(['images', 'type', 'annotations', 'categories'])
{'segmentation': [[155, 96, 155, 270, 351, 270, 351, 96]], 'area': 34104, 'iscrowd': 0, 'image_id': 12, 'bbox': [155, 96, 196, 174], 'category_id': 7, 'id': 1, 'ignore': 0} 
-----
{'file_name': '000012.jpg', 'height': 333, 'width': 500, 'id': 12} 
-----
[{'supercategory': 'none', 'id': 1, 'name': 'aeroplane'}, {'supercategory': 'none', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'none', 'id': 3, 'name': 'bird'}, {'supercategory': 'none', 'id': 4, 'name': 'boat'}, {'supercategory': 'none', 'id': 5, 'name': 'bottle'}, {'supercategory': 'none', 'id': 6, 'name': 'bus'}, {'supercategory': 'none', 'id': 7, 'name': 'car'}, {'supercategory': 'none', 'id': 8, 'name': 'cat'}, {'supercategory': 'none', 'id': 9, 'name': 'chair'}, {'supercategory': 'none', 'id': 10, 'name': 'cow'}, {'supercategory': 'none', 'id': 11, 'name': 'diningtable'}, {'supercategory': 'none', 'id': 12, 'name': 'dog'}, {'supercategory': 'none', 'id': 13, 'name': 'h

Seem to have some understanding of the structure of these labels, the annotations are key and then the image id is a lookup into the images.

Lookup from category ID to class name

In [16]:
id2name = {int(l['id']) : l['name'] for l in train_j[CATEGORIES]}
name2id = {l['name'] : int(l['id']) for l in train_j[CATEGORIES]}
cats = id2name

In [49]:
imid2name = {int(i['id']): i['file_name'] for i in train_j[IMAGES]}
name2imid = {i['file_name'] : int(i['id']) for i in train_j[IMAGES]}

Now have the training data, should probably convert bbox format to same as Jeremy, for consistancy. 

That is, x,y

In [46]:
def bb_voc2np(bb):
    return np.array([bb[1], bb[0], bb[3]+bb[1]-1, bb[2]+bb[0]-1])

def bb_np2voc(bb):
    return np.array([bb[1],bb[0],bb[3]-bb[1],bb[2]-bb[0]])

Let's get an array of dicts for training data

We want each dict to have 'image_id', 'bbox', 'category_id' and 'ingore' (pretty sure jeremy does something for ignore). These ignore flags represent harder detections where multiple instances are overlapping or the object is heavily occluded, i am pretty sure

In [26]:
def get_relevent_data(split_j, des_keys=['image_id', 'bbox', 'category_id']):
    # we do not want the ignores
    return [{key : datum[key] for key in des_keys} for datum in split_j[ANNOTATIONS] if not datum['ignore']]

def load_json_relevent(fname):
    jfile = json.load((PATH/fname).open())
    return get_relevent_data(jfile)

These functions create a list of the training, which is not what Jeremy does, he creates a dict with image_id as keys and bb with caterogy at items. So I will do the same

In [36]:
def get_data_dict(split_j):
    dat = collections.defaultdict(lambda:[])
    for l in split_j[ANNOTATIONS]:
        if not l['ignore']:
            bbox = bb_voc2np(l['bbox'])
            dat[l['image_id']].append((bbox, l['category_id']))
    return dat

train = get_data_dict(train_j)

Use "imid2name" to load one of the images using FastAI's open image function
